In [ ]:
import pulp
import networkx as nx
import itertools

In [ ]:
def pulp1(G):
   
    n = len(G.nodes)
    model = pulp.LpProblem("Metric_Dimension_Minimize_Set_Size", pulp.LpMinimize)

    y = pulp.LpVariable.dicts("y", G.nodes, cat="Binary")

    # Ciljna funkcija
    model += pulp.lpSum(y[j] for j in G.nodes), "Objective"

    # Ograničenja: Svaki par čvorova mora biti razrešen
    for u, v in itertools.combinations(G.nodes, 2):
        model += pulp.lpSum(abs(nx.shortest_path_length(G, u, j) - nx.shortest_path_length(G, v, j)) * y[j]
                             for j in G.nodes) >= 1, f"Resolve_{u}_{v}"
        
    model.solve(pulp.PULP_CBC_CMD(msg=False))

    # Prikaz rezultata
    resolving_set = [j for j in G.nodes if pulp.value(y[j]) == 1]
    return resolving_set

In [ ]:
def pulp2(G):
 
    n = len(G.nodes)

    # Početna gornja granica
    s = n
    best_resolving_set = None

    while s > 1:  # Iterativno smanjujemo s dok ne nađemo optimalno rešenje
        # Kreiranje ILP modela
        model = pulp.LpProblem("Metric_Dimension_Minimize_Unresolved_Pairs", pulp.LpMinimize)

        # Binarne promenljive y[j] = 1 ako je čvor j u rezolventnom skupu
        y = pulp.LpVariable.dicts("y", G.nodes, cat="Binary")

        # Binarne promenljive z[u, v] = 1 ako par (u, v) nije razrešen
        z = pulp.LpVariable.dicts("z", itertools.combinations(G.nodes, 2), cat="Binary")

        # Ciljna funkcija
        model += pulp.lpSum(z[u, v] for u, v in itertools.combinations(G.nodes, 2)), "Min_Unresolved_Pairs"

        # Ograničenje: kardinalnost skupa mora biti tačno s
        model += pulp.lpSum(y[j] for j in G.nodes) == s, "Fixed_Cardinality"

        # Ograničenja za razrešavanje parova
        for u, v in itertools.combinations(G.nodes, 2):
            model += pulp.lpSum(abs(nx.shortest_path_length(G, u, j) - nx.shortest_path_length(G, v, j)) * y[j]
                                 for j in G.nodes) + z[u, v] >= 1, f"Resolve_{u}_{v}"

        
        model.solve(pulp.PULP_CBC_CMD(msg=False))

        # Ako rešenje nije pronađeno, prekidamo petlju
        if pulp.LpStatus[model.status] != "Optimal":
            break

        # Uzimamo trenutno rešenje
        resolving_set = [j for j in G.nodes if pulp.value(y[j]) == 1]

        # Provera da li su svi parovi razrešeni
        unresolved_pairs = sum(pulp.value(z[u, v]) for u, v in itertools.combinations(G.nodes, 2))

        if unresolved_pairs == 0:
            # Ako nema nerešenih parova, čuvamo rešenje i pokušavamo da smanjimo s
            best_resolving_set = resolving_set
            s -= 1
        else:
            # Ako postoje nerešeni parovi, vraćamo najbolje do sada rešenje
            return best_resolving_set

    return best_resolving_set if best_resolving_set else []


In [ ]:
# G = nx.petersen_graph()
G = nx.Graph()
G.add_edges_from([(1,2),(1,3),(1,5),(1,6),(2,4),(2,6),(3,5),(3,4),(4,5),(4,6),(5,6)])

result1 = pulp1(G)
print("Optimalni rešavajući skup (Prvi model):", result1)

result2 = pulp2(G)
print("Optimalni rešavajući skup (Drugi model):", result2)

Optimalni rešavajući skup (Prvi model): [1, 3, 5]
Optimalni rešavajući skup (Drugi model): [1, 2, 3]
